#SEARCH USING COSINE SIMILARITY

In [3]:
import pandas as pd
import numpy as np
import os 
import re
import operator
import nltk 
import string
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
df = pd.read_csv('medicine_database_detailed.csv', engine='python', index_col =0)
df

,Medicine URL,Medicine Name,Category,Composition,Manufacturer,MRP,Uses,Side Effects
0,https://www.netmeds.com/prescriptions/atrest-1...,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur..."
1,https://www.netmeds.com/prescriptions/atrest-2...,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur..."
2,https://www.netmeds.com/prescriptions/capnea-i...,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...
3,https://www.netmeds.com/prescriptions/cognix-1...,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found
4,https://www.netmeds.com/prescriptions/a-cn-gel...,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or..."
...,...,...,...,...,...,...,...,...
23561,https://www.netmeds.com/prescriptions/wound-fi...,Wound Fix Solution 100ml,Wound,HYPOCHLOROUS ACID 0.006%+OXIDIZED WATER 99.97%,Strides Shasun Ltd,140.00,Data Not Found,Data Not Found
23562,https://www.netmeds.com/prescriptions/wounsol-...,Wounsol Ointment 15gm,Wound,Povidone Iodine+Sucralfate,Theia Health Care Pvt Ltd,75.00,Data Not Found,Data Not Found
23563,https://www.netmeds.com/prescriptions/zerosept...,ZEROSEPTIC Ointment 15gm,Wound,POVIDINE IODINE 5 %,Helios Pharmaceuticals,75.00,Data Not Found,Data Not Found
23564,https://www.netmeds.com/prescriptions/ds-gold-...,DS GOLD ANTI AGEING Cream 50gm,Wrinkle/Anti,SKINCARE PREPARATION,Adonis Laboratories Pvt Ltd,999.00,Data Not Found,Data Not Found


In [10]:
df1 = df.copy()

In [11]:
df1['text'] = df1['Medicine Name'] + ' ' +	df1['Category']	+ ' ' + df1['Composition'] + ' ' +	df1['Manufacturer']

In [12]:
df1 = df1.dropna()

In [13]:
df1.shape

(22998, 9)

In [14]:
df1 = df1.reset_index()

In [15]:
df1['cleaned_text'] = df1['text'].apply(lambda x: re.sub('(\d+(\.\d+)?)', r' \1 ', str(x)))

In [16]:
df1

,index,Medicine URL,Medicine Name,Category,Composition,Manufacturer,MRP,Uses,Side Effects,text,cleaned_text
0,0,https://www.netmeds.com/prescriptions/atrest-1...,Atrest 12.5mg Tablet 10'S,ADHD,TETRABENAZINE 12.5MG,Centaur Pharmaceuticals Pvt Ltd,146.01,ATREST 12.5MG is used in the treatment of the ...,"depression, suicidal thoughts or actions, neur...",Atrest 12.5mg Tablet 10'S ADHD TETRABENAZINE 1...,Atrest 12.5 mg Tablet 10 'S ADHD TETRABENAZI...
1,1,https://www.netmeds.com/prescriptions/atrest-2...,Atrest 25mg Tablet 10'S,ADHD,TETRABENAZINE 25MG,Centaur Pharmaceuticals Pvt Ltd,261.49,ATREST 25MG is used in the treatment of the in...,"depression, suicidal thoughts or actions, neur...",Atrest 25mg Tablet 10'S ADHD TETRABENAZINE 25M...,Atrest 25 mg Tablet 10 'S ADHD TETRABENAZINE...
2,2,https://www.netmeds.com/prescriptions/capnea-i...,Capnea Injection 1ml,ADHD,Caffeine Citrate 20 MG,Cipla Ltd,272.25,It is used in the treatment of interrupted bre...,serious intestinal disease with bloody stools ...,Capnea Injection 1ml ADHD Caffeine Citrate 20 ...,Capnea Injection 1 ml ADHD Caffeine Citrate ...
3,3,https://www.netmeds.com/prescriptions/cognix-1...,COGNIX 120mg Tablet 10's,ADHD,GINKGO BILOBA 120 mg,Eris Lifesciences Ltd,128.70,Data Not Found,Data Not Found,COGNIX 120mg Tablet 10's ADHD GINKGO BILOBA 12...,COGNIX 120 mg Tablet 10 's ADHD GINKGO BILOB...
4,4,https://www.netmeds.com/prescriptions/a-cn-gel...,A CN Gel(Topical) 20gm,Acne,CLINDAMYCIN 1%+NICOTINAMIDE 4%,Eskon Pharma,135.00,mild to moderate acne (spots),"dry skin, redness of the skin, skin burning or...",A CN Gel(Topical) 20gm Acne CLINDAMYCIN 1%+NIC...,A CN Gel(Topical) 20 gm Acne CLINDAMYCIN 1 %...
...,...,...,...,...,...,...,...,...,...,...,...
22993,23561,https://www.netmeds.com/prescriptions/wound-fi...,Wound Fix Solution 100ml,Wound,HYPOCHLOROUS ACID 0.006%+OXIDIZED WATER 99.97%,Strides Shasun Ltd,140.00,Data Not Found,Data Not Found,Wound Fix Solution 100ml Wound HYPOCHLOROUS AC...,Wound Fix Solution 100 ml Wound HYPOCHLOROUS ...
22994,23562,https://www.netmeds.com/prescriptions/wounsol-...,Wounsol Ointment 15gm,Wound,Povidone Iodine+Sucralfate,Theia Health Care Pvt Ltd,75.00,Data Not Found,Data Not Found,Wounsol Ointment 15gm Wound Povidone Iodine+Su...,Wounsol Ointment 15 gm Wound Povidone Iodine+...
22995,23563,https://www.netmeds.com/prescriptions/zerosept...,ZEROSEPTIC Ointment 15gm,Wound,POVIDINE IODINE 5 %,Helios Pharmaceuticals,75.00,Data Not Found,Data Not Found,ZEROSEPTIC Ointment 15gm Wound POVIDINE IODINE...,ZEROSEPTIC Ointment 15 gm Wound POVIDINE IODI...
22996,23564,https://www.netmeds.com/prescriptions/ds-gold-...,DS GOLD ANTI AGEING Cream 50gm,Wrinkle/Anti,SKINCARE PREPARATION,Adonis Laboratories Pvt Ltd,999.00,Data Not Found,Data Not Found,DS GOLD ANTI AGEING Cream 50gm Wrinkle/Anti SK...,DS GOLD ANTI AGEING Cream 50 gm Wrinkle/Anti ...


In [17]:
df1.iloc[0,10]

"Atrest  12.5 mg Tablet  10 'S ADHD TETRABENAZINE  12.5 MG Centaur Pharmaceuticals Pvt Ltd"

In [18]:
df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: str(x))
df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: x.lower())
df1['cleaned_text'] = df1['cleaned_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [19]:
def wordLemmatizer(data):
    tag_map = defaultdict(lambda : wn.NOUN)
    tag_map['J'] = wn.ADJ
    tag_map['V'] = wn.VERB
    tag_map['R'] = wn.ADV
    file_clean_k =pd.DataFrame()
    for index,entry in enumerate(data):
        
        # Declaring Empty List to store the words that follow the rules for this step
        Final_words = []
        # Initializing WordNetLemmatizer()
        word_Lemmatized = WordNetLemmatizer()
        # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
        for word, tag in pos_tag(entry):
            # Below condition is to check for Stop words and consider only alphabets
            if len(word)>1 and word not in stopwords.words('english') and word.isalpha():
                word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
                Final_words.append(word_Final)
            # The final processed set of words for each iteration will be stored in 'text_final'
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k.loc[index,'Keyword_final'] = str(Final_words)
                file_clean_k=file_clean_k.replace(to_replace ="\[.", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace ="'", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace =" ", value = '', regex = True)
                file_clean_k=file_clean_k.replace(to_replace ='\]', value = '', regex = True)
    return file_clean_k

In [20]:
df1['cleaned_text']

0        atrest  125 mg tablet  10 s adhd tetrabenazine...
1        atrest  25 mg tablet  10 s adhd tetrabenazine ...
2        capnea injection  1 ml adhd caffeine citrate  ...
3        cognix  120 mg tablet  10 s adhd ginkgo biloba...
4        a cn geltopical  20 gm acne clindamycin  1 nic...
                               ...                        
22993    wound fix solution  100 ml wound hypochlorous ...
22994    wounsol ointment  15 gm wound povidone iodines...
22995    zeroseptic ointment  15 gm wound povidine iodi...
22996    ds gold anti ageing cream  50 gm wrinkleanti s...
22997    versa gel  30 gm wrinkleanti cosmetics hibiscu...
Name: cleaned_text, Length: 22998, dtype: object

In [21]:
medicine = list(df1['cleaned_text'])

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
## Create Vocabulary
vocabulary = set()
for doc in medicine:
    vocabulary.update(doc.split(' '))
vocabulary = list(vocabulary)
# Intializating the tfIdf model
tfidf = TfidfVectorizer(vocabulary=vocabulary)
# Fit the TfIdf model
tfidf.fit(medicine)
# Transform the TfIdf model
tfidf_tran=tfidf.transform(medicine)

In [23]:
def gen_vector_T(tokens):
  Q = np.zeros((len(vocabulary)))    
  x= tfidf.transform(tokens)
  #print(tokens[0].split(','))
  for token in tokens[0].split(','):
      #print(token)
      try:
          ind = vocabulary.index(token)
          Q[ind]  = x[0, tfidf.vocabulary_[token]]
      except:
          pass
  return Q

In [24]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [25]:
def cosine_similarity_T(k, query):
    preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] =wordLemmatizer(q_df.q_clean)
    d_cosines = []
    
    query_vector = gen_vector_T(q_df['q_clean'])
    for d in tfidf_tran.A:
        d_cosines.append(cosine_sim(query_vector, d))
                    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    #print("")
    d_cosines.sort()
    a = pd.DataFrame()
    for i,index in enumerate(out):
        #a.loc[i,'index'] = str(index)
        a.loc[i,'Medicine'] = df1['Medicine Name'][index]
        a.loc[i, 'Manufacturer'] = df1['Manufacturer'][index]
        a.loc[i,'Uses'] = df1['Uses'][index]
        a.loc[i,'Side Effects'] = df1['Side Effects'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [26]:
text = '''['aa',
 '',
 't 4',
 '',
 '      ',
 '   ',
 '  ',
 '  ',
 ' ',
 '',
 'a tian ice : — k & en',
 'iii. 3 ae ; ‘ i nl is',
 'atig. Lic. No. : M/786/201 7 B22 No. Middlo Carrip,',
 'Net-10, East District,',
 '',
 ', Tei',
 '@® - Registered tracemark. Gangtok, Sikkim-737 135',
 '',
 '3 tan',
 '',
 '4)',
 '',
 '\\',
 ']',
 'Wy',
 '',
 '   ',
 '',
 '®',
 '',
 'ds',
 'y',
 '',
 'te',
 'ow',
 'oo Each film-coated tablet contains : | ee',
 '»” 625 mg Calcium Carbonate from an [Not to be Sold by leas',
 'Organic Source (Powdered Oyster Shell). sangre the Presuiinton | a,',
 '~~ equivalent to Elemental Calcium 250 mg Of a Registered MECCA ex',
 'ee Vitamin D3 1.P. 125 IU |Prattitions; es ©',
 ' —€a - Appropriate overages of vitamin added . 7 ee , ar a a Ag',
 '“to compensatetor.loss on storagece~ a POI T° SOLS.',
 '"ve = P ANAAAR ge. vv TORRENT. Zo',
 '',
 "> : lets ' SP et eS 3 : . oe ! ae",
 '~ Dosage : As:diteptest-by.the Physi: pHARMACEUTICALS LTD:',
 '',
 'sy',
 '> ~',
 '\\ ah 8',
 '. > aN',
 'dl “',
 'q',
 '~',
 '',
 ' ',
 '  ',
 '   ',
 '  ',
 ' ',
 ' ',
 ' ',
 ' ',
 '    ',
 ' ',
 ' ',
 '',
 '  ',
 '   ',
 '',
 '% a % . rw',
 'é‘ Yd',
 '; “ mp” MY',
 'eS as r ,',
 'Va |',
 '',
 '~, ~~ io',
 'i VN',
 'vw ™S',
 '',
 '  ',
 '',
 ' ',
 '       ',
 '\x0c']'''
#word_tokenize(text)
text = re.findall('\w+', text)
text = ' '.join(text)
text

'aa t 4 a tian ice k en iii 3 ae i nl is atig Lic No M 786 201 7 B22 No Middlo Carrip Net 10 East District Tei Registered tracemark Gangtok Sikkim 737 135 3 tan 4 Wy ds y te ow oo Each film coated tablet contains ee 625 mg Calcium Carbonate from an Not to be Sold by leas Organic Source Powdered Oyster Shell sangre the Presuiinton a equivalent to Elemental Calcium 250 mg Of a Registered MECCA ex ee Vitamin D3 1 P 125 IU Prattitions es a Appropriate overages of vitamin added 7 ee ar a a Ag to compensatetor loss on storagece a POI T SOLS ve P ANAAAR ge vv TORRENT Zo lets SP et eS 3 oe ae Dosage As diteptest by the Physi pHARMACEUTICALS LTD sy ah 8 aN dl q a rw é Yd mp MY eS as r Va io i VN vw S'

#TEST HERE

In [27]:
cosine_similarity_T(10, 'ranitidine')

,Medicine,Manufacturer,Uses,Side Effects,Score
0,Aciloc Injection 2ml,Cadila Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"any kind of skin rash or “hives”, flaking skin...",0.519403
1,Ranitin Injection 5X2ml,Torrent Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"any kind of skin rash or “hives”, flaking skin...",0.494715
2,Rantac Injection 5X2ml,J.B.Chemicals & Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"any kind of skin rash or “hives”, flaking skin...",0.490381
3,Rantac Syrup 100ml,J.B.Chemicals & Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"allergic reactions such as rash, itching or hi...",0.468942
4,ANTAC Injection 2ml,Zee Laboratories Ltd,Data Not Found,Data Not Found,0.464341
5,Rantop Syrup 100ml,Aristo Pharmaceuticals Pvt Ltd,It is used to heal and stop ulcers from bleedi...,"allergic reactions such as rash, itching or hi...",0.455115
6,R Loc Forte Tablet 15'S,Cadila Healthcare Ltd (Zydus),It is used to heal and stop ulcers from bleedi...,"allergic reactions include - rash, itching or ...",0.454361
7,H2Loc 40mg Injection 2ml,Torrent Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"any kind of skin rash or “hives”, flaking skin...",0.449125
8,Aciloc 150mg Tablet 30'S,Cadila Pharmaceuticals Ltd,It is used to heal and stop ulcers from bleedi...,"allergic reactions include - rash, itching or ...",0.444077
9,Bintac Injection 2ml,Biochem Pharmaceutical Industries Ltd,It is used to heal and stop ulcers from bleedi...,"any kind of skin rash or “hives”, flaking skin...",0.437085


In [ ]:
#  